## e011 EDA
 * startified_k_fold の妥当性調査

In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
import pyarrow.parquet as pq
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

import scipy
from scipy import signal
from scipy.signal import butter

from matplotlib import pyplot as plt
import seaborn as sns

import sys
sys.path.append('../../guchio_utils/')

In [2]:
#plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
# %config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 1

### Stratified の性能を見てみる

In [2]:
#trn_df = pd.read_parquet('../inputs/origin/train.parquet.zip', compression='zip')
trn_df = pq.read_pandas('../inputs/origin/train.parquet').to_pandas()
trn_meta_df = pd.read_csv('../inputs/origin/metadata_train.csv')

In [3]:
with open('../oofs/e011_2019-01-17-17-42-05_0.5739_0.311_oofs.pkl', 'rb') as fin:
    oofs = pickle.load(fin)

In [5]:
oofs[0]

[array([   0,    1,    2, ..., 1746, 1747, 1748]),
 array([1575, 1576, 1577, ..., 3487, 3488, 3489]),
 array([3372, 3373, 3374, ..., 5267, 5268, 5269]),
 array([4768, 4769, 4815, ..., 6993, 6994, 6995]),
 array([6654, 6655, 6656, ..., 8709, 8710, 8711])]

In [10]:
for labels in oofs[0]:
#    print(trn_meta_df.set_index('signal_id').loc[labels].target.sum())
    print(trn_meta_df.set_index('signal_id').loc[labels].id_measurement.nunique() * 3, trn_meta_df.set_index('signal_id').loc[labels].reset_index().signal_id.nunique())

1743 1743
1746 1743
1752 1742
1749 1742
1743 1742


## e012 EDA
 * 各 oof の threshold から single model で submit したほうが良いのかを考える (普通 auc は rank avg) (model 毎に threshold は違うはず) 

In [2]:
#trn_df = pd.read_parquet('../inputs/origin/train.parquet.zip', compression='zip')
trn_df = pq.read_pandas('../inputs/origin/train.parquet').to_pandas()
trn_meta_df = pd.read_csv('../inputs/origin/metadata_train.csv')

In [11]:
with open('../oofs/e008_2019-01-17-08-04-32_0.5673_0.139_oofs.pkl', 'rb') as fin:
    oofs = pickle.load(fin)

In [13]:
df_elems = []
for i, oof in enumerate(zip(oofs[0], oofs[1])):
    signal_id, y_pred = oof
    _df = pd.DataFrame()
    _df['signal_id'] = signal_id
    _df['y_pred'] = y_pred
    _df['fold'] = i
    df_elems.append(_df)

oof_df = pd.concat(df_elems, axis=0)
oof_df

,signal_id,y_pred,fold
0,3,0.507173,0
1,4,0.273839,0
2,5,0.338976,0
3,18,0.000015,0
4,19,0.000823,0
5,20,0.005122,0
6,33,0.000007,0
7,34,0.000046,0
8,35,0.000093,0
9,48,0.015403,0


In [15]:
trn_meta_oof_df = trn_meta_df.merge(oof_df, on='signal_id', how='left')
trn_meta_oof_df

,signal_id,id_measurement,phase,target,y_pred,fold
0,0,0,0,0,0.012684,3
1,1,0,1,0,0.008258,3
2,2,0,2,0,0.025044,3
3,3,1,0,1,0.507173,0
4,4,1,1,1,0.273839,0
5,5,1,2,1,0.338976,0
6,6,2,0,0,0.000416,4
7,7,2,1,0,0.000609,4
8,8,2,2,0,0.000498,4
9,9,3,0,0,0.000204,3


In [16]:
sys.path.append('../tools/utils/')
from metrics import calc_best_MCC

In [19]:
# threshold 全然違うやんけ...
for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    best_MCC, best_thresh = calc_best_MCC(mcc_df.target, mcc_df.y_pred)
    print(best_MCC, best_thresh)

100%|██████████| 30/30 [00:00<00:00, 682.27it/s]

0.5817122285912858 0.10424137931034483
0.5273301399042729 0.5860344827586207
0.5952264597279228 0.20748275862068966
0.6276506515467083 0.6548620689655172
0.6300460299735168 0.34513793103448276


In [20]:
# 単に concat したら threshold はこんな感じ
calc_best_MCC(trn_meta_oof_df.target, trn_meta_oof_df.y_pred)

100%|██████████| 30/30 [00:00<00:00, 593.41it/s]


(0.567333198083413, 0.1386551724137931)

In [52]:
from metrics import calc_MCC

In [54]:
# こんなかんじで本当に良くなる...？
# あ、でも test 時には変わるか
for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    MCC = calc_MCC(mcc_df.target, mcc_df.y_pred, thresh=0.1386551724137931)
    print(MCC)

0.5736031038490991
0.4760015946100805
0.561058906688545
0.5897363271372071
0.6218053132183833


In [55]:
# bins 上げると...？
calc_best_MCC(trn_meta_oof_df.target, trn_meta_oof_df.y_pred, bins=3000)

100%|██████████| 3000/3000 [00:05<00:00, 568.58it/s]


(0.5702475839248895, 0.13411103701233743)

In [35]:
# rank avg してみる
# trn_meta_oof_df = trn_meta_oof_df.set_index('signal_id')
trn_meta_oof_df['ranked_preds'] = 0.

for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    trn_meta_oof_df.loc[mcc_df.index, 'ranked_preds'] = ((mcc_df.y_pred.rank() - 1) / (mcc_df.shape[0] - 1))
    
trn_meta_oof_df   

,id_measurement,phase,target,y_pred,fold,ranked_preds
signal_id,,,,,,
0,0,0,0,0.012684,3,0.882319
1,0,1,0,0.008258,3,0.868542
2,0,2,0,0.025044,3,0.903559
3,1,0,1,0.507173,0,0.965557
4,1,1,1,0.273839,0,0.956372
5,1,2,1,0.338976,0,0.959816
6,2,0,0,0.000416,4,0.633698
7,2,1,0,0.000609,4,0.680276
8,2,2,0,0.000498,4,0.657274


In [42]:
# rank avg したら threshold 悪くなったぞ...
for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    best_MCC, best_thresh = calc_best_MCC(mcc_df.target, mcc_df.ranked_preds)
    print(best_MCC, best_thresh)

100%|██████████| 30/30 [00:00<00:00, 733.98it/s]

0.5806245298518179 0.9301724137931034
0.48543450491005197 0.9645862068965517
0.5622267333313686 0.9645862068965517
0.6161803114136581 0.9645862068965517
0.6119923581769747 0.9645862068965517


In [49]:
# bins をちょうせいすればましになった
for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    best_MCC, best_thresh = calc_best_MCC(mcc_df.target, mcc_df.ranked_preds, bins=3000)
    print(best_MCC, best_thresh)

  3%|▎         | 78/3000 [00:00<00:03, 777.19it/s]

0.5850331044971155 0.9410966988996331


  2%|▏         | 74/3000 [00:00<00:03, 739.08it/s]

0.5273301399042729 0.97537279093031


  2%|▏         | 72/3000 [00:00<00:04, 717.41it/s]

0.5952264597279228 0.9547405801933977


  2%|▏         | 71/3000 [00:00<00:04, 701.20it/s]

0.6276854003058253 0.9603977992664221


100%|██████████| 3000/3000 [00:04<00:00, 669.88it/s]

0.6300460299735168 0.9593994664888296


In [50]:
# 結局 bin の値に敏感になってるきがする
calc_best_MCC(trn_meta_oof_df.target, trn_meta_oof_df.ranked_preds, bins=3000)

100%|██████████| 3000/3000 [00:05<00:00, 590.52it/s]


(0.568471124988459, 0.9547405801933977)

In [51]:
# bins をちょうせいすればましになった
for fold in range(5):
    mcc_df = trn_meta_oof_df.query(f'fold == {fold}')
    MCC = calc_MCC(mcc_df.target, mcc_df.ranked_preds, thresh=0.9547405801933977)
    print(MCC)

0.5437837740064833
0.4724930871318122
0.5952264597279228
0.614717199923815
0.6172836636520752


In [59]:
e008_df = pd.read_csv('../submissions/e008_2019-01-17-08-04-32_0.5673_0.139_sub.csv.gz', compression='gzip')
e008_df = pd.read_csv('../submissions/e008_2019-01-17-08-04-32_0.5673_0.139_sub.csv.gz', compression='gzip')
e012_df = pd.read_csv('../submissions/e012_2019-01-17-22-47-11_0.5947_sub.csv.gz', compression='gzip')

In [63]:
pd.concat([e008_df.target, e012_df.target], axis=1).corr()

,target,target
target,1.000000,0.754826
target,0.754826,1.000000


In [64]:
e008_df.target.sum(), e012_df.target.sum()

(978, 569)

In [65]:
e008_df.shape

(20337, 2)